### Lab Assignment: Commercial Data Analysis

### University of Virginia
### DS 7200: Distributed Computing
### Last Updated: August 20, 2023

---

### INSTRUCTIONS  
In this assignment, you will work with a dataset containing information about businesses.  
Each record is a business location.  Follow the steps below, writing and running the code in blocks, and displaying the solutions.  

Each question part is worth 1 POINT, for a total of 15 POINTS.

Hint: reaching deeper fields in json hierarchy can be done like this:  

`df.select('address.street_number')`

---

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
        .appName("comm") \
        .getOrCreate()

# import data types
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

/opt/conda/lib/python3.7/site-packages/pyspark/bin/load-spark-env.sh: line 68: ps: command not found
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/09/16 16:13:02 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
# note that read.json can read a zipped JSON directly

**1. (1 PT) Read in the dataset and show the number of records**

In [3]:
df = spark.read.json('/project/ds5559/large_datasets/part-00000-a159c41a-bc58-4476-9b78-c437667f9c2b-c000.json.gz')
df.count()

23/09/13 14:35:16 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


154679

**2. (1 PT) Show the first 5 records**

In [4]:
df.show(5)

+--------------------+--------------------+--------------------+----------------+----+--------------------+--------------------+--------------------+
|             address|       business_tags|               hours|              id|menu|             reviews|                urls|             webpage|
+--------------------+--------------------+--------------------+----------------+----+--------------------+--------------------+--------------------+
|{Woodburn, {45.15...|                null|                null|000023995a540868|null|                  []|{woodburn.k12.or....|{Educational Tech...|
|{Hialeah, {25.884...|{[], [{has_atm, Y...|{null, 1900, null...|0000821a1394916e|null|                null|{null, [yelp.com]...|                null|
|{Rochester, {43.1...|{[], [{accepts_cr...|{null, 1700, null...|000136e65d50c3b7|null|[{New (to me) qui...|{usps.com, [yelp....|{Welcome | USPS G...|
|{West Palm Beach,...|                null|                null|00014329a70b9869|null|              

**3. (1 PT) Show the first 5 street addresses which are not null**  

In [5]:
df.filter(df['address.street_address'].isNotNull()).select(df['address.street_address']).show(5)

+-------------------+
|     street_address|
+-------------------+
| Cooper Contracting|
|          Route 607|
|Bush St & Kearny St|
|          S 14th St|
|             Rr 474|
+-------------------+
only showing top 5 rows



**4. (1 PT) Location**  

Count the number of records where the city is Phoenix

In [6]:
df.filter(df['address.city']=='Phoenix').count()

762

**5. (1 PT) Hours**  

Count the number of records where closing time on Thursday is 8pm

In [7]:
df.filter(df['hours.thursday_close']=='2000').count()

3313

**6. (1 PT) Location and Hours**  

Count the number of records where city is Phoenix and closing time on Thursday is 8pm

In [8]:
df.filter((df['hours.thursday_close']=='2000') & (df['address.city']=='Phoenix')).count()

12

**7. (1 PT) Price Range**  

Price range is quoted in number of dollar signs.  Count the number of records with price range greater than or equal to two.

In [9]:
df.filter(df['menu.price_range']>=2).count()

1135

**8. (1 PT) COMPANY HEADQUARTERS**  

For the `address.is_headquarters` field:  
how many locations are HQ / are NOT HQ / are null?

In [10]:
not_hq = df.filter(df['address.is_headquarters']=='false').count()
hq = df.filter(df['address.is_headquarters']=='true').count()
null = df.filter(df['address.is_headquarters'].isNull()).count()
print(f"There are {hq} HQs, {not_hq} not HQs, and {null} null values")

There are 318 HQs, 66736 not HQs, and 87625 null values


**9. (1 PT) Webpage URLs**  

Register the dataframe as a temp table.  
Next, use Spark SQL to select only the webpage title column, filtering on rows where the webpage url (accessed under `webpage.url`) is *Target.com*. 

Show only one resulting row and don't truncate the output.

In [11]:
df.createOrReplaceTempView("data")

# query the view
sqlDF = spark.sql("SELECT webpage.title FROM data WHERE webpage.url == 'Target.com'")
sqlDF.show(1, False)

+-------------------------------+
|title                          |
+-------------------------------+
|Target : Expect More. Pay Less.|
+-------------------------------+
only showing top 1 row



**10. (1 PT) Analysis on Ratings**  

The reviews contains information such as the number of stars for each review (the *rating*).  
The ratings are stored in an array (`reviews.stars`) for each business location (you should check for yourself). Return the top five most common rating arrays.  For example, an array might look like: 
[5, 5]



In [12]:
sqlDF = spark.sql('''SELECT reviews.stars, count(reviews.stars) AS review_counts 
                  FROM data 
                  GROUP BY reviews.stars 
                  ORDER BY review_counts
                  DESC LIMIT 5''')
sqlDF.show()

+------+-------------+
| stars|review_counts|
+------+-------------+
|    []|        42419|
|   [5]|         4258|
|[null]|         3067|
|[5, 5]|         1610|
|   [1]|         1559|
+------+-------------+



**11. More work with Ratings**  

For this question, you will filter out null ratings and then compute the average rating for each business location (using the field: `id`).


a) (1 PT) Create a new dataframe retaining two fields: `id`, `reviews.stars`


In [26]:
df2 = df.select(df['id'], df['reviews.stars'])
df2.show()

+----------------+--------------------+
|              id|               stars|
+----------------+--------------------+
|000023995a540868|                  []|
|0000821a1394916e|                null|
|000136e65d50c3b7|              [4, 4]|
|00014329a70b9869|                null|
|00031c0a83f00657|                null|
|0003b7589a4e12a0|                 [5]|
|00045f958e4bb02a|[null, null, null...|
|00055eb14ea38ca2|                null|
|00059519f0dba1b4|[null, null, null...|
|0005f8f0b2beeac4|                null|
|000673b2dac411ea|                null|
|0006d5aa170bae22|                  []|
|00080a1e9b3607c0|                null|
|0008bc70f8ba62bf|              [null]|
|0008c39401d7ce3b|                null|
|0009cd9a574331a9|                null|
|000a1df4c8e0ecd2|[null, null, 4, 5...|
|000a863f1d02dd78|                null|
|000b4a9059161ee7|                null|
|000bf1e934ac9cb6|                  []|
+----------------+--------------------+
only showing top 20 rows



b) (1 PT) Create a row for each rating  
hint: use the `withColumn()` and `explode()` functions  
you will need to import the `explode()` function by issuing:

`from pyspark.sql.functions import explode`


In [29]:
from pyspark.sql.functions import explode
df2 = df2.withColumn("stars", explode(df2.stars))
df2.show()

+----------------+-----+
|              id|stars|
+----------------+-----+
|000136e65d50c3b7|    4|
|000136e65d50c3b7|    4|
|0003b7589a4e12a0|    5|
|00045f958e4bb02a| null|
|00045f958e4bb02a| null|
|00045f958e4bb02a| null|
|00045f958e4bb02a| null|
|00059519f0dba1b4| null|
|00059519f0dba1b4| null|
|00059519f0dba1b4| null|
|00059519f0dba1b4| null|
|00059519f0dba1b4| null|
|00059519f0dba1b4| null|
|00059519f0dba1b4| null|
|00059519f0dba1b4| null|
|00059519f0dba1b4|    1|
|00059519f0dba1b4|    5|
|00059519f0dba1b4|    2|
|00059519f0dba1b4|    4|
|00059519f0dba1b4|    5|
+----------------+-----+
only showing top 20 rows



c) (1 PT) Return a count of the number of ratings in this dataframe

In [30]:
df2.count()

600082

d) (1 PT) Drop rows where the rating is null, and return a count of the number of non-null ratings

In [34]:
df2 = df2.filter(df2['stars'].isNotNull())
df2.count()

538241

e) (1 PT) Compute the average rating, grouped by `id`. After the average is computed, sort by `id` in ascending order and show the top 10 records.  
 
hint:   
this can all be done in one line using the `agg()` function  
this `id` should be at the top: 000136e65d50c3b7

In [46]:
from pyspark.sql import functions as F
from pyspark.sql.functions import asc
df2.groupBy(df2['id']).agg(F.avg(df2['stars'])).sort(asc("id")).show(10)

+----------------+------------------+
|              id|        avg(stars)|
+----------------+------------------+
|000136e65d50c3b7|               4.0|
|0003b7589a4e12a0|               5.0|
|00059519f0dba1b4|3.3333333333333335|
|000a1df4c8e0ecd2|               4.6|
|000c7b7a30623083|               5.0|
|000c9ffc8b89af03|               3.0|
|000de20baa847ecc|1.6666666666666667|
|001064359d9f162f|               5.0|
|0010c9f495d87dd7|               3.0|
|0017774db5e6400a| 4.333333333333333|
+----------------+------------------+
only showing top 10 rows

